In [1]:
client_id = "e5a7fb4e507f4b83a942647e75961e84"
client_secret = "bea3ad93c3a64d34acf4fe4410fdeb5c"

# core

## universal imports n methods

In [2]:
import base64
import requests
import json


## run this for no flask -- can't do user shit

In [3]:

def get_token():
    auth_string = f"{client_id}:{client_secret}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type" : "client_credentials"
    }

    result = requests.post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    return json_result['access_token']

def get_auth_header(token):
    return {"Authorization" : "Bearer " + token}

# COMMON VARIABLES
ub_url = "https://api.spotify.com/v1"

In [4]:
token = get_token()

## run this when flask is also running - can do user shit

In [25]:
from urllib.parse import quote
import webbrowser

redirect_uri = "http://localhost:8888/callback"

def get_authcode():
    encoded_rduri = quote(redirect_uri)

    scopes = [
        "user-follow-read",
        "user-top-read",
        "user-read-private",
        "user-read-email"
    ]


    encoded_scopes = quote((' '.join(scopes)).strip())

    auth_url = f"https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri={encoded_rduri}&scope={encoded_scopes}"
    webbrowser.open(auth_url)

    print("After granting permission, you will be redirected to your redirect URI with a code in the query string.")
    print("Paste the full redirect URI here: ")
    redirected_uri = input()

    # Extract the code from the redirected URI
    code = redirected_uri.split("code=")[1].split("&")[0]

    return code

def get_token_authcode(code):
    auth_string = f"{client_id}:{client_secret}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type" : "authorization_code",
        "code": code,
        "redirect_uri" : redirect_uri
    }
    result = requests.post(url, headers = headers, data = data)
    json_result = result.json()
    return json_result['access_token']

In [5]:
ub_url = "https://api.spotify.com/v1"
code = get_authcode()
token = get_token_authcode(code)


NameError: name 'get_authcode' is not defined

## funsies (2nd order functions lmao)

In [6]:
def search_for_artist(token, artist_name):
    url = f"{ub_url}/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1" # limit=1 -- first result only

    query_url = f"{url}{query}"

    result = requests.get(query_url, headers = headers)
    json_result = result.json()["artists"]["items"]

    if len(json_result) == 0:
        print("search failed bitch")
        return None
    return json_result[0]
    

def get_songs_by_artist(token, artist_id):
    url = f"{ub_url}/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = result.json()["tracks"]
    return json_result

def search_for_track(token, track_name):
    url = f"{ub_url}/search"
    headers = get_auth_header(token)
    query = f"?q={track_name}&type=track&limit=1"
    query_url = f"{url}{query}"

    result = requests.get(query_url, headers = headers)
    json_result = result.json()
    return json_result['tracks']['items'][0]


def get_track_audio_features(token, track_id):
    url = f"{ub_url}/audio-features/{track_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = result.json()
    return json_result


## "3rd order functions" xD

In [7]:
def get_bpm_key(token, track_id):
    audio_features = get_track_audio_features(token, track_id)
    bpm = audio_features['tempo']

    key_pitch_class = audio_features['key']
    to_notes = {
        0: "C",
        1: "C#/Db",
        2: "D",
        3: "D#/Eb",
        4: "E",
        5: "F",
        6: "F#/Gb",
        7: "G",
        8: "G#/Ab",
        9: "A",
        10: "A#/Bb",
        11: "B"
    }
    key = to_notes.get(key_pitch_class)

    return bpm, key

* ideas: 

# TEMPLATE

# REAL PLAYGROUND

## get key and bpm

In [8]:
trackres = search_for_track(token, input("song? "))
id = trackres['id']
name = trackres['name']

bpm, key = get_bpm_key(token, id)
print(f"{name} : {bpm} BPM {key}")

Bags : 104.996 BPM C#/Db


In [10]:
# https://developer.spotify.com/documentation/web-api/reference/get-recommendations

In [9]:
get_track_audio_features(token, id)

{'danceability': 0.742,
 'energy': 0.546,
 'key': 1,
 'loudness': -7.694,
 'mode': 1,
 'speechiness': 0.0315,
 'acousticness': 0.172,
 'instrumentalness': 0.38,
 'liveness': 0.115,
 'valence': 0.868,
 'tempo': 104.996,
 'type': 'audio_features',
 'id': '6X3FZtz4cKU2MKSQlGG9ZG',
 'uri': 'spotify:track:6X3FZtz4cKU2MKSQlGG9ZG',
 'track_href': 'https://api.spotify.com/v1/tracks/6X3FZtz4cKU2MKSQlGG9ZG',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6X3FZtz4cKU2MKSQlGG9ZG',
 'duration_ms': 260520,
 'time_signature': 4}

In [ ]:
def template(token):
    # the url to send requests.get to
    url = f"{ub_url}/audio-features/2TmqHjg7uhizGndzXQdFuf"

    # headers
    headers = get_auth_header(token)

    # the requests.get
    result = requests.get(url, headers = headers)

    #json code and return
    json_result = result.json()
    return json_result
    

# USER CONTENT

In [27]:
def get_top_tracks(token, limit, sml):
    timerange = "short_term" if sml.lower() == 's' else "medium_term" if sml.lower() == 'm' else "long_term"
    url = f"{ub_url}/me/top/tracks?limit={limit}&time_range={timerange}"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = result.json()
    return json_result

def get_top_artists(token, limit, sml):
    timerange = "short_term" if sml.lower() == 's' else "medium_term" if sml.lower() == 'm' else "long_term"
    url = f"{ub_url}/me/top/artists?limit={limit}&time_range={timerange}"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = result.json()
    return json_result

In [29]:
print(tt)

{'error': {'status': 400, 'message': 'Invalid time range'}}


In [30]:
tt = get_top_tracks(token, 50, "short_term")

for idx, t in enumerate(tt['items']):
    artists = []
    for i in t['album']['artists']:
        artists.append(i['name'])
    print(f"{idx + 1}. {' '.join(artists[n] for n, _ in enumerate(artists))} - {t['name']}")

1. dose - hidden track (pxe)
2. Ginseng - M16
3. my bloody valentine - Off Your Face
4. Sickboyrari - Edward Scissorhandz
5. my bloody valentine - thorn
6. LSD and the Search for God - This Time
7. Bladee - Requiem
8. my bloody valentine - good for you
9. Deftones - Rx Queen
10. Kiryano - Emo Plugg
11. RXKNephew - Take Three
12. Mazzy Star - Be My Angel
13. my bloody valentine - emptiness inside
14. The Smashing Pumpkins - The Boy
15. my bloody valentine - cigarette in your bed
16. Mojave 3 - Love Songs on the Radio
17. The Sundays - Goodbye
18. julie - flutter
19. Sonic Youth - Unmade Bed
20. Elliott Smith - Say Yes
21. Rooster - Rented Starship
22. Cocteau Twins - Fotzepolitic
23. RXKNephew - Skyline Tony Too Good
24. Varg²™ Thaiboy Digital - Hitty
25. Blue Smiley - dark
26. The Smashing Pumpkins - Luna - 2011 Remaster
27. Starflyer 59 - Do You Ever Feel This Way - Gold Album Version
28. my bloody valentine - if i am
29. Sonic Youth - Shadow of a Doubt
30. RXKNephew - Rxk Please Hurt

In [13]:
ta = get_top_artists(token, 50)

for idx, t in enumerate(ta['items']):
    print(f"{idx + 1}. {t['name']}")

1. Sonic Youth
2. Bladee
3. LUCKI
4. Sickboyrari
5. my bloody valentine
6. Yung Lean
7. Title Fight
8. Kiryano
9. RXKNephew
10. beabadoobee
11. Parannoul
12. Crystal Castles
13. Panchiko
14. meat computer
15. d1v
16. Drain Gang Archive
17. BETWEEN FRIENDS
18. Snow Strippers
19. Slow Pulp
20. A$AP Rocky
21. Yves Tumor
22. Drop Nineteens
23. The Smashing Pumpkins
24. Blue Smiley
25. Chinese Football
26. Thaiboy Digital
27. Larry June
28. Hole
29. Macabre Plaza
30. PinkPantheress
31. what is your name?
32. Ecco2k
33. BBY GOYARD
34. Earl Sweatshirt
35. Hardrock
36. Della Zyr
37. yeule
38. J Dilla
39. UnoTheActivist
40. Nujabes
41. Xavier Wulf
42. Starflyer 59
43. Pale Saints
44. Men I Trust
45. Sampha
46. Jeff Rosenstock
47. The Cure
48. Asian Glow
49. The Sundays
50. Pity Sex


In [12]:

an_authcode = "AQAsBWqSuPgu6xI16r1t8mYvvG9wM68bGaacts6w4oGh2RNw87--6Kytzed0_Q7SNDTsm8Yulr3v7u2DX3JDwo25LyOV1UBQSQYz9RgFPiCjHELZViMRUXBNHOcLn7q2035teGOy-5zniIfKgQC_saUD_YWteeVZ4MGXesTJSjn-gL6lHmMCgxkUfEg6tE-765ddasIoq4WYP4Jn_dv-eZBP5SVuQ87WGvab26uUWe8gMSAvaInfBqYiTAZkml-IYTrbKhrBxg"

token = get_token_authcode(an_authcode)

KeyError: 'access_token'

In [12]:
code = get_authcode()

After granting permission, you will be redirected to your redirect URI with a code in the query string.
Paste the full redirect URI here: 


In [10]:
p = '''James Ivy - Birthday
Bladee & Mechatok - God (Evian Christ Remix) [Kelsey Park Edit]
454 - THEY KNOW
A.G Cook - Windows (No Rome Remix)
James Ivy - ID
@jimmy2_ - one last time <3
PinkPantheress - Just For Me
James Ivy & Wavedash - ID
James Ivy - ID
James Ivy - ID
James Ivy - Texas (Early Demo Version)
1000000000s - Effective Dream
Lil Uzi Vert - Run It Up
Woesum & Glasear - Notice Me
Weiland - Cry
Nijiko - Tinypiano
@jimmy2_ - fireball
Emotegi - Under My Skin (James Ivy Blink Redo)
CFCF - Punk Song
Andrew Todd - Welcome2
vierre cloud - moment
@jimmy2_ - hide n seek
James Ivy - My God! It's Killing Me
James Ivy - My God! It's Killing Me (Reprise)'''

l = p.split('\n')

for song in l:
    tr = search_for_track(token, song)
    print(f"{tr['name']} - {tr['artists'][0]['name']}")

Involved - James Ivy
Be Nice 2 Me - Bladee
Strip Club Music - KE4T
Windows - No Rome Remix - A. G. Cook
Ivy - IdiOtBuRNs
One Last Time - Ariana Grande
Just for Me - PinkPantheress
Sick - James Ivy
Ivy - IdiOtBuRNs
Ivy - IdiOtBuRNs
Texas - James Ivy
10,000 Reasons (Bless The Lord) [Radio Version] - Radio Version/Live - Matt Redman
XO Tour Llif3 - Lil Uzi Vert
Summer Rain - Woesum
Demons - Weiland
Nijiko (Tiny Piano) - Masakatsu Takagi
Bottle of Wine (Mono 45 Mix) - Jimmy Gilmer & The Fireballs
Under My Skin - Nate Smith
You Hear Colours - CFCF
ANDREW TATE ON DISCIPLINE - Motivation Hub
moment - Vierre Cloud
Games of Hide and Seek - Jimmy Sky
Killing Me - Waynewood
My God Is Number One - Yancy
